#Data Preprocessing

Published Documents:
 - [Spotify Podcasts Dataset](https://arxiv.org/pdf/2004.04270v3.pdf)
 - [Speech Recognition Diarization](https://arxiv.org/pdf/2005.08072.pdf)
 - [Unsupervised Topic Segmentation of Meetings with BERT Embeddings](https://arxiv.org/pdf/2106.12978.pdf)


Dataset:
 - [This American Life Podcast Transcripts](https://www.kaggle.com/datasets/shuyangli94/this-american-life-podcast-transcriptsalignments?resource=download)

Citation:

 - Mao, H. H., Li, S., McAuley, J., & Cottrell, G. (2020). Speech Recognition and Multi-Speaker Diarization of Long Conversations. INTERSPEECH.

## Imported Packages and Libraries

In [1]:
!pip install nlp --quiet

     |████████████████████████████████| 1.7 MB 5.6 MB/s 
     |████████████████████████████████| 212 kB 42.3 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
name = "John"
if name == 'Ricardo':
  path = '/content/drive/MyDrive/MIDS/W266/w266_nlp_project/
else:
  path = ''

In [ ]:
from collections import Counter
import numpy as np
import tensorflow as tf
from tensorflow import keras
from nlp import load_dataset

import seaborn as sns
from pprint import pprint 

from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import json

# Formatting options for float number in numpy
float_formatter = "{:.4f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

## The American Life Podcast Dataset

### Load

In [ ]:
#dataset_path = '/content/drive/MyDrive/MIDS/W266/w266_nlp_project'
dataset_path = '/content/drive/MyDrive/MIDS/W266/w266_nlp_project/TALDataset' #  rmarin

In [ ]:
full_speaker_data = 'full-speaker-map.json'
test_transcript_data = 'test-transcripts-aligned.json'
train_transcript_data = 'train-transcripts-aligned.json'
valid_transcript_data = 'valid-transcripts-aligned.json'

In [ ]:
ep_num = 11

In [ ]:
path_train = os.path.join(dataset_path,train_transcript_data)
f = open(path_train,)
train_dataset = json.load(f)

NameError: ignored

In [ ]:
pprint(train_dataset.get('ep-1')[10])

### Process

In [ ]:
#train_dataset.keys()
S = []            #  list of M utterances S = {S_1,..., S_M}
T = []            #  Underlying topic structure Ti ∈ [Sj , Sk].
Y = []            #  Label sequence Y = {y1,.., yM} yi is binary indicates whether the utterance Si is the start of a new topic segment
topic_label = []  #  Topic label
speaker = []      #  Speaker as in  epsiode.role
speaker_name = [] #  Speaker name as in episode.speaker
episode = []      #  Episode Number
alignments_T = [] #  Alignments correspondant to topic structure

topic_1 = ""  
idx_1 = 0    
utterance_start_1 = 0
for idx, i in enumerate(train_dataset['ep-1']):
  S.append(i['utterance'])
  topic = i['act']
  topic_label.append(topic)
  speaker.append(i['role'])
  speaker_name.append(i['speaker'])
  episode.append(i['episode'])
  if topic != topic_1:
    T.append((idx_1,idx))
    alignments_T.append((utterance_start_1,i['utterance_start']))
    Y.append(1)
    idx_1 = idx
    utterance_start_1 = i['utterance_start']
  else:
    Y.append(0)
  topic_1 = topic


#train_dataset['ep-1'][0]
#train_dataset['ep-1'][0]['role']
#pprint(train_dataset.get('ep-1')[10])
print(Y)
print(S)
print(T)
print(topic_label)
print(speaker)
print(speaker_name)
print(episode)
print(alignments_T)


## Stuff You Should Know Podcast Dataset

### Load

In [ ]:
f = open('/content/drive/MyDrive/W266 Final Project/datasets/SYSK/sysk_with_transcripts_10 Accidental Inventions By the Numbers.json')

sysk_train_dataset = json.load(f)

In [ ]:
pprint(sysk_train_dataset.get('text'))

("I was in the at and T store for an upgrade. I left with at and T's best deal "
 'on a smartphone and a choice of plan. But on my way out, here comes this new '
 'guy, a noncarrier phone and a plan that raised eyebrows. I felt for him when '
 'I tell you we left the store store grinning from ear to ear with the same '
 "deal. I love watching people prosper. You feel me? That's when I learned "
 "that whether you joined today or have been with at and T for years, they'll "
 'have the same best deals for everyone on every smartphone. Eligible plan '
 'required. Offers vary by device. Restrictions may apply. See att. '
 'Comdealsfordetails. Hey, everybody. If you want a great website, you want to '
 'do it yourself with no Muslim fuss, turn to Squarespace. They have '
 'everything to sell anything. They have the tools that you need to get your '
 'business off the ground, including ecommerce templates, inventory '
 "management, simple checkout process, and secure payments. And if you're 